In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import os

In [2]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Define the model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

In [7]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 790us/step - accuracy: 0.8781 - loss: 0.4275 - val_accuracy: 0.9577 - val_loss: 0.1465
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - accuracy: 0.9649 - loss: 0.1235 - val_accuracy: 0.9684 - val_loss: 0.1031
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 722us/step - accuracy: 0.9766 - loss: 0.0788 - val_accuracy: 0.9723 - val_loss: 0.0899
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 716us/step - accuracy: 0.9827 - loss: 0.0567 - val_accuracy: 0.9734 - val_loss: 0.0856
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 712us/step - accuracy: 0.9870 - loss: 0.0437 - val_accuracy: 0.9740 - val_loss: 0.0814


In [9]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 0s - 444us/step - accuracy: 0.9740 - loss: 0.0814

Test accuracy: 0.9739999771118164


In [10]:
# Save the model
model_dir = '../app/models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
model.save(os.path.join(model_dir, 'mnist_model.h5'))
print('Model saved!')

Model saved!
